# 2024衆議院選挙における世論地図のデータ

In [ ]:
# Import necessary libraries
%pip install --upgrade pip jupyter
%pip install pandas
import pandas as pd

In [2]:
zid_to_topic = {
    26: "政治資金規制改革",
    28: "税制",
    29: "社会保障（年金・医療）",
    30: "経済政策・物価高対策",
    31: "エネルギー政策",
    34: "憲法",
    38: "デジタル民主主義"
}
topic_to_zid = {
    "政治資金規制改革": 26,
    "税制": 28,
    "社会保障（年金・医療）": 29,
    "経済政策・物価高対策": 30,
    "エネルギー政策": 31,
    "憲法": 34,
    "デジタル民主主義": 38
}

In [3]:
df = pd.read_csv("votes.csv")
matrix = df.pivot_table(index="pid", columns=["zid", "tid"], values="vote")
matrix

zid    26                                      28       ...   38            \
tid    0    1    2    3    4    5    6    7    0    1   ...   10   11   12   
pid                                                     ...                  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     1.0  1.0  1.0  1.0 -1.0 -1.0  1.0 -1.0  1.0  1.0  ... -1.0  1.0 -1.0   
2     0.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  0.0  0.0  ...  1.0 -1.0 -1.0   
3     1.0  0.0  0.0 -1.0  1.0  1.0 -1.0 -1.0  1.0  1.0  ...  1.0  1.0 -1.0   
4     1.0 -1.0 -1.0 -1.0  1.0  1.0  0.0 -1.0  0.0 -1.0  ... -1.0 -1.0 -1.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
4398  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4399  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4400  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4401  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   
4402  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN   

zid                                      
tid    13   14   15   16   17   18   19  
pid                                      
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1    -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  
2    -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  
3    -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  
4    -1.0 -1.0  0.0  1.0 -1.0 -1.0  1.0  
...   ...  ...  ...  ...  ...  ...  ...  
4398  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4399  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4400  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4401  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4402  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[4403 rows x 84 columns]

In [4]:
comment_df = pd.read_csv("comments.csv")[["tid", "zid", "txt"]]
comment_df


,tid,zid,txt
0,0,38,行政手続きをすべてオンライン化する
1,0,26,政治資金パーティーはオンライン含めて開催を禁止する
2,1,26,企業・団体によるパーティー券購入は禁止する
3,2,26,企業・団体献金は禁止する
4,3,26,政策活動費は即時廃止する
...,...,...,...
79,15,38,マイナンバーによる行政のデータ連携を加速し全銀行口座との紐づけを実現することで、所得のリアル...
80,16,38,警察によるウェアラブルカメラの運用を広げ、治安維持対応を強化する
81,17,38,行政手続きにAIを導入し、補助金申請の審査業務などは人間からAIに代替させていく
82,18,38,年末調整は廃止し、国民全員がワンストップで確定申告が可能な行政インフラを整える


In [5]:
matrix_keizai = matrix[topic_to_zid["経済政策・物価高対策"]]
matrix_keizai

tid,0,1,2,3,4,5,6
pid,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
2,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
3,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
4,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...
4398,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
4399,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0
4400,0.0,-1.0,0.0,1.0,0.0,-1.0,1.0


In [6]:
comment_keizai = comment_df[comment_df["zid"] == topic_to_zid["経済政策・物価高対策"]]
comment_keizai

,tid,zid,txt
38,0,30,低所得者世帯を下支えする給付金を支給する
39,1,30,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続
40,2,30,最低賃金を1500円以上に引き上げる
41,3,30,物価高対策として、消費税を引き下げる、又は廃止する
42,4,30,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる
43,5,30,再エネ賦課金の徴収停止により電気代を引き下げる
44,6,30,中小企業の価格転嫁を支援し、賃上げを実現する


In [7]:
import pandas as pd

# 集計コード
result = pd.DataFrame({
    'count_0': (matrix_keizai == 0).sum(axis=0),
    'count_1': (matrix_keizai == 1).sum(axis=0),
    'count_-1': (matrix_keizai == -1).sum(axis=0),
    'count_NaN': matrix_keizai.isna().sum(axis=0)
})

# 結果を表示
result


,count_0,count_1,count_-1,count_NaN
tid,,,,
0,1162,1111,1351,698
1,693,433,2536,663
2,1158,577,1961,625
3,700,949,2040,648
4,965,323,2405,649
5,1104,431,2102,690
6,617,166,2900,672


In [8]:
# 各tidに対応するtxtを一度に取得し、resultに新しい列として追加
result["txt"] = result.index.map(lambda tid: comment_keizai.loc[comment_keizai["tid"] == tid, "txt"].values[0])
result

,count_0,count_1,count_-1,count_NaN,txt
tid,,,,,
0,1162,1111,1351,698,低所得者世帯を下支えする給付金を支給する
1,693,433,2536,663,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続
2,1158,577,1961,625,最低賃金を1500円以上に引き上げる
3,700,949,2040,648,物価高対策として、消費税を引き下げる、又は廃止する
4,965,323,2405,649,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる
5,1104,431,2102,690,再エネ賦課金の徴収停止により電気代を引き下げる
6,617,166,2900,672,中小企業の価格転嫁を支援し、賃上げを実現する


In [9]:
# count_0, count_1, count_-1の合計を計算
total_counts = result[['count_0', 'count_1', 'count_-1']].sum(axis=1)
result['N'] = total_counts

# 各値を合計で割ってパーセントに変換
result['賛成(%)'] = (result['count_1'] / total_counts * 100).round(1)
result['中立(%)'] = (result['count_0'] / total_counts * 100).round(1)
result['反対(%)'] = (result['count_-1'] / total_counts * 100).round(1)

# パーセント表記の列を表示
result[['txt', '賛成(%)', '中立(%)', '反対(%)', 'N']]


,txt,賛成(%),中立(%),反対(%),N
tid,,,,,
0,低所得者世帯を下支えする給付金を支給する,30.7,32.1,37.3,3624
1,電気・ガス代、燃料費高騰対策のため、政府による補助金施策を継続,11.8,18.9,69.3,3662
2,最低賃金を1500円以上に引き上げる,15.6,31.3,53.1,3696
3,物価高対策として、消費税を引き下げる、又は廃止する,25.7,19.0,55.3,3689
4,トリガー条項凍結又は二重課税廃止によるガソリン減税により、ガソリン代を引き下げる,8.7,26.1,65.1,3693
5,再エネ賦課金の徴収停止により電気代を引き下げる,11.9,30.4,57.8,3637
6,中小企業の価格転嫁を支援し、賃上げを実現する,4.5,16.8,78.7,3683
